In [1]:
%load_ext line_profiler
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import pyxel.inputs_outputs as io
import numpy as np
from pyxel.run import single_mode

/System/Volumes/Data/work/sw/pyxel/pyxel/inputs_outputs/configuration.py:20: RuntimeWarning: Cannot import 'pygmo
  from pyxel.calibration import Algorithm, Calibration


# Current arctic model
## Load file `Arctic_1.yaml` with 'old' arctic model

In [3]:
# Create a configuration class from the yaml file
arctic_config = io.load("Arctic_1.yaml")

arctic_single = arctic_config.single
arctic_detector = arctic_config.ccd_detector
arctic_pipeline = arctic_config.pipeline


# Display current 'arctic_pipeline'
arctic_pipeline.charge_transfer.arctic

ModelFunction(name='arctic', func=pyxel.models.charge_transfer.arctic.arctic, arguments=Arguments({'well_fill_power': 0.8, 'density': 100, 'release_timescale': 1.2, 'express': 20}), enabled=True)

In [4]:
# Change parameter 'express'
arctic_pipeline.charge_transfer.arctic.arguments["express"] = 40

## Benchmark with 'old' arctic model

In [5]:
%%timeit

single_mode(single=arctic_single, detector=arctic_detector, pipeline=arctic_pipeline)

5.16 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# New arctic model
## Modify pipeline with 'new' arctic model

In [6]:
from pyxel.models.charge_transfer import arctic_fast

# Hack
arctic_pipeline.charge_transfer.arctic._func = arctic_fast

# Display current 'arctic_pipeline'
arctic_pipeline.charge_transfer.arctic

ModelFunction(name='arctic', func=pyxel.models.charge_transfer.arctic_fast.arctic_fast, arguments=Arguments({'well_fill_power': 0.8, 'density': 100, 'release_timescale': 1.2, 'express': 40}), enabled=True)

## Benchmark with 'new' arctic model

In [7]:
%%timeit

single_mode(single=arctic_single, detector=arctic_detector, pipeline=arctic_pipeline)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor
/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor
/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor
/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor
/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = 

2.44 s ± 30.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Benchmarks line by line

### Function `arctic_fast`

In [8]:
%lprun -f arctic_fast single_mode(arctic_single, arctic_detector, arctic_pipeline)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor


Timer unit: 1e-06 s

Total time: 5.00137 s
File: /System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py
Function: arctic_fast at line 961

Line #      Hits         Time  Per Hit   % Time  Line Contents
   961                                           def arctic_fast(
   962                                               detector: CCD,
   963                                               well_fill_power: float,
   964                                               density: float,
   965                                               release_timescale: t.Sequence[float],
   966                                               express=0,
   967                                           ) -> np.ndarray:
   968         1         10.0     10.0      0.0      char = detector.characteristics
   969         1          4.0      4.0      0.0      image = detector.pixel.array
   970         1          9.0      9.0      0.0      image = image.astype(float)
   971         1         

### Function `add_cti`

In [9]:
from pyxel.models.charge_transfer.arctic_fast import add_cti

%lprun -f add_cti single_mode(arctic_single, arctic_detector, arctic_pipeline)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor


Timer unit: 1e-06 s

Total time: 4.9828 s
File: /System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py
Function: add_cti at line 864

Line #      Hits         Time  Per Hit   % Time  Line Contents
   864                                           def add_cti(
   865                                               image,
   866                                               parallel_ccd=None,
   867                                               parallel_roe=None,
   868                                               parallel_traps=None,
   869                                               parallel_express=0,
   870                                               parallel_offset=0,
   871                                               parallel_window_range=None,
   872                                               serial_ccd=None,
   873                                               serial_roe=None,
   874                                               serial_traps=None,
 

### Function `clock_charge_in_one_direction`

In [10]:
from pyxel.models.charge_transfer.arctic_fast import clock_charge_in_one_direction

%lprun -f clock_charge_in_one_direction single_mode(arctic_single, arctic_detector, arctic_pipeline)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor


Timer unit: 1e-06 s

Total time: 5.8423 s
File: /System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py
Function: clock_charge_in_one_direction at line 695

Line #      Hits         Time  Per Hit   % Time  Line Contents
   695                                           def clock_charge_in_one_direction(
   696                                               image_in,
   697                                               ccd,
   698                                               roe,
   699                                               traps,
   700                                               express,
   701                                               offset,
   702                                               window_row_range,
   703                                               window_column_range,
   704                                               time_window_range,
   705                                           ):
   706                                    

### Function `func_n_electrons_released_and_captured`

In [11]:
from pyxel.models.charge_transfer.arctic_fast import (
    func_n_electrons_released_and_captured,
)

%lprun -f func_n_electrons_released_and_captured single_mode(arctic_single, arctic_detector, arctic_pipeline)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor


Timer unit: 1e-06 s

Total time: 4.28608 s
File: /System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py
Function: func_n_electrons_released_and_captured at line 545

Line #      Hits         Time  Per Hit   % Time  Line Contents
   545                                           def func_n_electrons_released_and_captured(
   546                                               self,  # type: TrapManager
   547                                               n_free_electrons,
   548                                               ccd_filling_function,
   549                                               dwell_time=1,
   550                                               express_multiplier=1,
   551                                           ):
   552                                           
   553                                               # Initial watermarks and number of electrons in traps
   554                                               # watermarks_initial = d

### Function `_n_electrons_released_and_captured_cloud_below_watermarks`

In [12]:
from pyxel.models.charge_transfer.arctic_fast import (
    _n_electrons_released_and_captured_cloud_below_watermarks,
)

%lprun -f _n_electrons_released_and_captured_cloud_below_watermarks single_mode(arctic_single, arctic_detector, arctic_pipeline)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py:282: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty = capture_rates * exponential_factor


Timer unit: 1e-06 s

Total time: 1.25151 s
File: /System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_fast.py
Function: _n_electrons_released_and_captured_cloud_below_watermarks at line 336

Line #      Hits         Time  Per Hit   % Time  Line Contents
   336                                           def _n_electrons_released_and_captured_cloud_below_watermarks(
   337                                               self,
   338                                               n_free_electrons,
   339                                               cloud_fractional_volume,
   340                                               watermarks_initial,
   341                                               watermark_index_above_cloud,
   342                                               max_watermark_index,
   343                                               fill_probabilities_from_release,
   344                                               n_trapped_electrons_initial,
   345     